In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [3]:
#read in data
permits_df = pd.read_csv("../data/original_data/sidewalk-cafe-permits.csv")
#drop duplicates but there are none
permits_df.drop_duplicates()
#Rename columns to what we want to be in SQL Database
permits_df = permits_df.rename(columns={"PERMIT NUMBER": "permit_number", "DOING BUSINESS AS NAME": "business_name", "ISSUED DATE": "issued_date", "EXPIRATION DATE": "expiration_date", "ADDRESS": "address", "LATITUDE": "latitude", "LONGITUDE": "longitude", "LOCATION": "location"})

In [4]:
#Changed Column names
permits_df.columns

Index(['permit_number', 'ACCOUNT NUMBER', 'SITE NUMBER', 'LEGAL NAME',
       'business_name', 'issued_date', 'expiration_date', 'PAYMENT DATE',
       'address', 'ADDRESS NUMBER START', 'ADDRESS NUMBER', 'STREET DIRECTION',
       'STREET', 'STREET TYPE', 'CITY', 'STATE', 'ZIP CODE', 'WARD',
       'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 'latitude', 'longitude',
       'location', 'Community Areas', 'Zip Codes', 'Boundaries - ZIP Codes',
       'Census Tracts', 'Wards'],
      dtype='object')

In [5]:
# Make dataframe of only the columns we want to bring into SQL
import_permits_df = permits_df[["permit_number", "business_name", "issued_date", "expiration_date", "address", "latitude", "longitude"]]


In [6]:
#Find any null values
import_permits_df.isnull().values.any()
#drop the null values
import_permits_df = import_permits_df.dropna()


In [7]:
import_permits_df.head()

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1142666,TINTO & TAPAS,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,7958 W BELMONT AVE,41.937859,-87.826255
1,1142290,Pie cafe,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,5357 N ASHLAND AVE,41.979717,-87.669565
2,1142132,KALIFLOWER,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,333 N MICHIGAN AVE,41.887543,-87.624407
3,1142101,SMALL CHEVAL,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,1732 N MILWAUKEE AVE,41.912730,-87.681327
4,1142100,THE WHALE CHICAGO,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2427-2431 N MILWAUKEE AVE,41.925388,-87.701170


In [9]:
#Read in second CSV file which contains data for 2020
current_permits = pd.read_csv("../data/original_data/current_permits.csv")
current_permits.head()
#Rename columns to bring into SQL
current_permits = current_permits.rename(columns={"PERMIT NUMBER": "permit_number", "DOING BUSINESS AS NAME": "business_name", "ISSUED DATE": "issued_date", "EXPIRATION DATE": "expiration_date", "ADDRESS": "address", "LATITUDE": "latitude", "LONGITUDE": "longitude", "LOCATION": "location"})


In [10]:
#Create import Dataframe with only columns we want to bring into SQL
import_current_permits_df = current_permits[["permit_number", "business_name", "issued_date", "expiration_date", "address", "latitude", "longitude"]]
import_current_permits_df


,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1143796,TURTLES,02/27/2020,05/31/2021,238 W 33RD ST,41.834685,-87.633095
1,1143980,HOTEL EMC2,04/10/2020,05/31/2021,228 E ONTARIO ST,41.893483,-87.621931
2,1144014,PIAZZA BELLA OSTERIA,02/27/2020,05/31/2021,2114 W ROSCOE ST,41.943289,-87.681528
3,1143772,AU CHEVAL,03/04/2020,05/31/2021,800 W RANDOLPH ST,41.884515,-87.647497
4,1144011,SPACE519,05/22/2020,05/31/2021,200 E CHESTNUT ST,41.898530,-87.621777
...,...,...,...,...,...,...,...
1090,1148345,FRANKLIN ROOM,04/09/2021,02/28/2022,675 N FRANKLIN ST,41.894589,-87.635506
1091,1148368,Kraken Sushi & Beyond,04/13/2021,02/28/2022,4022 N WESTERN AVE,41.954759,-87.688635
1092,1148330,AVEC II,04/13/2021,02/28/2022,141 W ERIE ST,41.893900,-87.632732
1093,1148380,FONS EMPANADAS,04/13/2021,02/28/2022,2566 N CLARK ST,41.929403,-87.642899


In [11]:
#find any null values 
import_current_permits_df.isnull().sum()
#drop the null values
import_current_permits_df = import_current_permits_df.dropna()

In [12]:
import_current_permits_df.head()

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1143796,TURTLES,02/27/2020,05/31/2021,238 W 33RD ST,41.834685,-87.633095
1,1143980,HOTEL EMC2,04/10/2020,05/31/2021,228 E ONTARIO ST,41.893483,-87.621931
2,1144014,PIAZZA BELLA OSTERIA,02/27/2020,05/31/2021,2114 W ROSCOE ST,41.943289,-87.681528
3,1143772,AU CHEVAL,03/04/2020,05/31/2021,800 W RANDOLPH ST,41.884515,-87.647497
4,1144011,SPACE519,05/22/2020,05/31/2021,200 E CHESTNUT ST,41.898530,-87.621777


In [13]:
# After creating the 2 tables in posgresql (sql) via pgAdmin 4:
# Connect to local database
##connection_string = "postgres:Hinsdale1@localhost:5432/chicago_permits"
#engine = create_engine(f'postgresql://{connection_string}')
# # ensure table names appear
 #engine.table_names()


#engine = create_engine("sqlite:///chicago_permits.sqlite")

In [14]:
# Load past_permits dataframe into sql database
#import_permits_df.to_sql(name='past_permits', con=engine, if_exists='append', index=False)

In [15]:
# Load current_permits dataframe into sql database
#import_current_permits_df.to_sql(name='current_permits', con=engine, if_exists='append', index=False)

In [16]:
import_current_permits_df
import_permits_df

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1142666,TINTO & TAPAS,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,7958 W BELMONT AVE,41.937859,-87.826255
1,1142290,Pie cafe,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,5357 N ASHLAND AVE,41.979717,-87.669565
2,1142132,KALIFLOWER,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,333 N MICHIGAN AVE,41.887543,-87.624407
3,1142101,SMALL CHEVAL,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,1732 N MILWAUKEE AVE,41.912730,-87.681327
4,1142100,THE WHALE CHICAGO,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2427-2431 N MILWAUKEE AVE,41.925388,-87.701170
...,...,...,...,...,...,...,...
17993,1003612,THE REVEL ROOM,2001-08-01T00:00:00.000,2001-11-01T00:00:00.000,1566 N MILWAUKEE AVE,41.909880,-87.676914
17994,1002993,O'NEIL'S BAR & GRILL,2001-04-04T00:00:00.000,2001-11-01T00:00:00.000,152 E ONTARIO ST,41.893460,-87.623374
17995,1002652,CAFE MAURO,2001-05-07T00:00:00.000,2001-11-01T00:00:00.000,3633 W 26TH ST,41.844259,-87.715946
17996,1001307,Rick's Cafe-Casablanca,2001-06-18T00:00:00.000,2001-11-01T00:00:00.000,3915 N SHERIDAN RD,41.953158,-87.654359


In [17]:
# Confirm data has been added by querying joined table
pd.read_sql_query('''
select * from current_permits as c
union
select * from past_permits as p;
''', con=engine)

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1000571,THE 5 BOROUGHS DELICATESSEN,2001-04-03T00:00:00.000,2001-11-01T00:00:00.000,738 N WELLS ST,41.895920,-87.634330
1,1001307,Rick's Cafe-Casablanca,2001-06-18T00:00:00.000,2001-11-01T00:00:00.000,3915 N SHERIDAN RD,41.953158,-87.654359
2,1002652,CAFE MAURO,2001-05-07T00:00:00.000,2001-11-01T00:00:00.000,3633 W 26TH ST,41.844259,-87.715946
3,1002993,O'NEIL'S BAR & GRILL,2001-04-04T00:00:00.000,2001-11-01T00:00:00.000,152 E ONTARIO ST,41.893460,-87.623374
4,1003612,THE REVEL ROOM,2001-08-01T00:00:00.000,2001-11-01T00:00:00.000,1566 N MILWAUKEE AVE,41.909880,-87.676914
...,...,...,...,...,...,...,...
19085,1148333,LITTLE WOK,04/07/2021,02/28/2022,3144 N BROADWAY,41.939052,-87.644513
19086,1148345,FRANKLIN ROOM,04/09/2021,02/28/2022,675 N FRANKLIN ST,41.894589,-87.635506
19087,1148367,GRACIE O'MALLEY'S EDGEWATER,04/14/2021,02/28/2022,6334 N CLARK ST,41.997373,-87.670549
19088,1148368,Kraken Sushi & Beyond,04/13/2021,02/28/2022,4022 N WESTERN AVE,41.954759,-87.688635


In [36]:
import_current_permits_df['year'] = pd.DatetimeIndex( import_current_permits_df['issued_date']).year
##temp_15_data = import_current_permits_df[temp_transformed['year']== 2015]
##temp_15_data.head

<ipython-input-36-85b97a5bb218>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import_current_permits_df['year'] = pd.DatetimeIndex( import_current_permits_df['issued_date']).year


In [37]:
import_current_permits_df.head()

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude,year
0,1143796,TURTLES,02/27/2020,05/31/2021,238 W 33RD ST,41.834685,-87.633095,2020
1,1143980,HOTEL EMC2,04/10/2020,05/31/2021,228 E ONTARIO ST,41.893483,-87.621931,2020
2,1144014,PIAZZA BELLA OSTERIA,02/27/2020,05/31/2021,2114 W ROSCOE ST,41.943289,-87.681528,2020
3,1143772,AU CHEVAL,03/04/2020,05/31/2021,800 W RANDOLPH ST,41.884515,-87.647497,2020
4,1144011,SPACE519,05/22/2020,05/31/2021,200 E CHESTNUT ST,41.898530,-87.621777,2020


In [38]:
import_permits_df['year'] = pd.DatetimeIndex( import_permits_df['issued_date']).year

In [39]:
import_permits_df.head()

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude,year
0,1142666,TINTO & TAPAS,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,7958 W BELMONT AVE,41.937859,-87.826255,2019
1,1142290,Pie cafe,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,5357 N ASHLAND AVE,41.979717,-87.669565,2019
2,1142132,KALIFLOWER,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,333 N MICHIGAN AVE,41.887543,-87.624407,2019
3,1142101,SMALL CHEVAL,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,1732 N MILWAUKEE AVE,41.912730,-87.681327,2019
4,1142100,THE WHALE CHICAGO,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2427-2431 N MILWAUKEE AVE,41.925388,-87.701170,2019


In [18]:
#Create a dataframe that includes origin counts for a bar graph 
Years_df_pivot=import_permits_df.groupby(["issued_date", "permit_number"])["issued_date"].agg("count").to_frame("issued_date_counts").reset_index()
Years_df_pivot

,issued_date,permit_number,issued_date_counts
0,2001-03-14T00:00:00.000,1007419,1
1,2001-03-15T00:00:00.000,1007416,1
2,2001-03-19T00:00:00.000,1007411,1
3,2001-03-20T00:00:00.000,1007406,1
4,2001-03-20T00:00:00.000,1007417,1
...,...,...,...
17991,2019-10-28T00:00:00.000,1138700,1
17992,2019-11-07T00:00:00.000,1140922,1
17993,2019-11-07T00:00:00.000,1141641,1
17994,2019-11-15T00:00:00.000,1140177,1


In [22]:
pd.to_datetime(Years_df_pivot["issued_date"]).dt.year

0        2001
1        2001
2        2001
3        2001
4        2001
         ... 
17991    2019
17992    2019
17993    2019
17994    2019
17995    2019
Name: issued_date, Length: 17996, dtype: int64

In [23]:
Years_df_pivot["year"]=pd.to_datetime(Years_df_pivot["issued_date"]).dt.year

In [24]:
Years_df_pivot

,issued_date,permit_number,issued_date_counts,year
0,2001-03-14T00:00:00.000,1007419,1,2001
1,2001-03-15T00:00:00.000,1007416,1,2001
2,2001-03-19T00:00:00.000,1007411,1,2001
3,2001-03-20T00:00:00.000,1007406,1,2001
4,2001-03-20T00:00:00.000,1007417,1,2001
...,...,...,...,...
17991,2019-10-28T00:00:00.000,1138700,1,2019
17992,2019-11-07T00:00:00.000,1140922,1,2019
17993,2019-11-07T00:00:00.000,1141641,1,2019
17994,2019-11-15T00:00:00.000,1140177,1,2019


In [27]:
Years_df_pivot.groupby("year")["issued_date"].count()

year
2001     482
2002     551
2003     637
2004     698
2005     721
2006     758
2007     823
2008     875
2009     919
2010     981
2011    1042
2012    1097
2013    1158
2014    1192
2015    1195
2016    1220
2017    1228
2018    1224
2019    1195
Name: issued_date, dtype: int64

In [29]:
#Create a dataframe that includes origin counts for a bar graph 
Years_df_two_pivot=import_current_permits_df.groupby(["issued_date", "permit_number"])["issued_date"].agg("count").to_frame("issued_date_counts").reset_index()
Years_df_two_pivot

,issued_date,permit_number,issued_date_counts
0,01/27/2020,1143686,1
1,02/04/2021,1144991,1
2,02/09/2021,1145676,1
3,02/10/2020,1143664,1
4,02/10/2020,1143676,1
...,...,...,...
1089,12/14/2020,1145365,1
1090,12/15/2020,1145872,1
1091,12/17/2020,1145562,1
1092,12/21/2020,1144464,1


In [30]:
pd.to_datetime(Years_df_two_pivot["issued_date"]).dt.year

0       2020
1       2021
2       2021
3       2020
4       2020
        ... 
1089    2020
1090    2020
1091    2020
1092    2020
1093    2020
Name: issued_date, Length: 1094, dtype: int64

In [31]:
Years_df_two_pivot["year"]=pd.to_datetime(Years_df_two_pivot["issued_date"]).dt.year

In [32]:
Years_df_two_pivot

,issued_date,permit_number,issued_date_counts,year
0,01/27/2020,1143686,1,2020
1,02/04/2021,1144991,1,2021
2,02/09/2021,1145676,1,2021
3,02/10/2020,1143664,1,2020
4,02/10/2020,1143676,1,2020
...,...,...,...,...
1089,12/14/2020,1145365,1,2020
1090,12/15/2020,1145872,1,2020
1091,12/17/2020,1145562,1,2020
1092,12/21/2020,1144464,1,2020


In [33]:
Years_df_two_pivot.groupby("year")["issued_date"].count()

year
2020    1064
2021      30
Name: issued_date, dtype: int64

In [35]:
# Connect to local database
connection_string = "postgres:Hinsdale1@localhost:5432/chicago_permits"
engine = create_engine(f'postgresql://{connection_string}')
# # ensure table names appear
engine.table_names()

['past_permits', 'current_permits']

In [40]:
import_permits_df.to_sql(name='past_permits', con=engine, if_exists='append', index=False)

In [41]:
import_current_permits_df.to_sql(name='current_permits', con=engine, if_exists='append', index=False)